<a href="https://colab.research.google.com/github/SeyfGoumeida/Time_series_Unsupervised_Learning/blob/main/unsupervised_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd

In [26]:
acce_x = pd.read_csv('acce_x.txt', sep=" ", header=None)
acce_y = pd.read_csv('acce_y.txt', sep=" ", header=None)
acce_z = pd.read_csv('acce_z.txt', sep=" ", header=None)
accm_x = pd.read_csv('accm_x.txt', sep=" ", header=None)
accm_y = pd.read_csv('accm_y.txt', sep=" ", header=None)
accm_z = pd.read_csv('accm_z.txt', sep=" ", header=None)
vit_x = pd.read_csv('vit_x.txt', sep=" ", header=None)
vit_y = pd.read_csv('vit_y.txt', sep=" ", header=None)
vit_z = pd.read_csv('vit_z.txt', sep=" ", header=None)
lab = pd.read_csv('lab.txt', sep=" ", header=None)
for i in range(0,len(acce_x)):
  acce_x.rename(columns={i: 'acce_x'+str(i)} ,inplace=True)
  acce_y.rename(columns={i: 'acce_y'+str(i)} ,inplace=True)
  acce_z.rename(columns={i: 'acce_z'+str(i)} ,inplace=True)
  accm_x.rename(columns={i: 'accm_x'+str(i)} ,inplace=True)
  accm_y.rename(columns={i: 'accm_y'+str(i)} ,inplace=True)
  accm_z.rename(columns={i: 'accm_z'+str(i)} ,inplace=True)
  vit_x.rename(columns={i: 'vit_x'+str(i)} ,inplace=True)
  vit_y.rename(columns={i: 'vit_y'+str(i)} ,inplace=True)
  vit_z.rename(columns={i: 'vit_z'+str(i)} ,inplace=True)
  lab.rename(columns={i: 'lab'+str(i)} ,inplace=True)
  

In [24]:
frames = [acce_x, acce_y,acce_z,accm_x,accm_y,accm_z,vit_x,vit_y,vit_z,lab]
df = pd.concat(frames,axis=1)
df

,acce_x0,acce_x1,acce_x2,acce_x3,acce_x4,acce_x5,acce_x6,acce_x7,acce_x8,acce_x9,acce_x10,acce_x11,acce_x12,acce_x13,acce_x14,acce_x15,acce_x16,acce_x17,acce_x18,acce_x19,acce_x20,acce_x21,acce_x22,acce_x23,acce_x24,acce_x25,acce_x26,acce_x27,acce_x28,acce_x29,acce_x30,acce_x31,acce_x32,acce_x33,acce_x34,acce_x35,acce_x36,acce_x37,acce_x38,acce_x39,...,vit_z89,vit_z90,vit_z91,vit_z92,vit_z93,vit_z94,vit_z95,vit_z96,vit_z97,vit_z98,vit_z99,vit_z100,vit_z101,vit_z102,vit_z103,vit_z104,vit_z105,vit_z106,vit_z107,vit_z108,vit_z109,vit_z110,vit_z111,vit_z112,vit_z113,vit_z114,vit_z115,vit_z116,vit_z117,vit_z118,vit_z119,vit_z120,vit_z121,vit_z122,vit_z123,vit_z124,vit_z125,vit_z126,vit_z127,lab0
0,0.000181,0.010139,0.009276,0.005066,0.010810,0.004045,0.004757,0.006214,0.003307,0.007572,0.005407,0.006221,0.006895,0.004610,0.007331,0.005078,0.005763,0.006693,0.005443,0.008241,0.006506,0.006532,0.007422,0.005772,0.006240,0.004623,0.004833,0.005499,0.004341,0.005098,0.004269,0.003045,0.003204,0.004520,0.005127,0.003550,0.004234,0.004469,0.003573,0.005136,...,0.011268,0.014257,0.014786,0.014159,0.008384,0.001907,0.004132,0.007871,0.007843,0.008190,0.007086,0.005538,0.005446,0.004286,0.003560,0.002002,-0.000017,0.002038,0.003736,0.003148,0.005214,0.007802,0.006516,0.004947,0.006479,0.006764,0.006273,0.005763,-0.000024,-0.006042,-0.006891,-0.004903,0.001354,0.008033,0.007355,0.002669,-0.002170,-0.005643,-0.001446,5
1,0.001094,0.004550,0.002879,0.002247,0.003305,0.002416,0.001619,0.000981,0.000009,-0.000363,-0.000487,-0.000356,-0.000229,-0.000131,-0.000441,-0.001565,-0.000929,0.000009,-0.001236,-0.000911,-0.000435,-0.001177,0.000165,0.000751,0.000389,0.001328,0.002053,0.002738,0.002743,0.001330,-0.000367,-0.000722,-0.001904,-0.004294,-0.000025,0.005251,0.002490,0.000811,0.001166,-0.000693,...,-0.006524,-0.007353,-0.009759,-0.004647,0.002527,0.008457,0.011938,0.009866,0.009631,0.011067,0.009372,0.006185,0.001905,0.000190,-0.000414,-0.004500,-0.005280,-0.000161,0.005794,0.009647,0.009009,0.008349,0.010212,0.010737,0.012291,0.015449,0.020503,0.025133,0.020913,0.016544,0.018556,0.018173,0.018046,0.019393,0.018070,0.014628,0.005045,-0.004424,-0.002798,5
2,0.003531,0.002285,-0.000420,-0.003738,-0.006706,-0.003148,0.000733,0.000668,0.002162,-0.000946,-0.006476,-0.003423,-0.000610,-0.002929,-0.001796,0.000956,0.002311,0.002538,0.001180,0.001217,0.000869,0.000686,0.002156,0.000478,-0.001506,-0.002105,-0.001494,0.002181,0.002876,0.000871,-0.000392,-0.000569,0.000569,-0.001176,-0.001957,-0.000471,0.000208,0.003380,0.004829,0.003380,...,-0.005931,-0.000026,-0.002708,-0.002540,-0.003191,-0.007055,-0.005317,-0.006089,-0.002329,0.003786,-0.001737,-0.005536,-0.003725,-0.004714,-0.006092,-0.004016,0.002862,0.011245,0.013716,0.010446,0.010813,0.010439,0.006563,0.007488,0.004233,-0.002323,-0.002383,-0.003674,-0.004933,-0.003351,-0.002691,-0.000283,0.000404,-0.001556,-0.001239,-0.003032,-0.005566,-0.003702,-0.002645,5
3,-0.001772,-0.001311,0.000388,0.000408,-0.000355,0.000998,0.001109,-0.003149,-0.008882,-0.010483,-0.004482,0.004528,0.008167,0.002929,-0.004487,-0.004717,-0.001637,-0.000097,0.001614,0.002619,0.004765,0.005851,0.002579,0.000677,0.002138,0.003519,0.002715,0.002370,0.003299,0.002357,0.001481,-0.000340,-0.002201,0.000124,0.002345,0.002706,0.001281,-0.000949,0.000890,0.004952,...,0.001323,0.001530,-0.002582,-0.004664,-0.004718,-0.006203,-0.008205,-0.011258,-0.011302,-0.010440,-0.012088,-0.009789,-0.008616,-0.008832,-0.001896,0.001873,-0.001333,-0.002344,-0.001665,-0.001346,-0.001697,-0.003776,-0.005683,-0.004502,-0.002338,-0.002227,-0.002969,-0.005610,-0.008606,-0.009409,-0.011208,-0.013223,-0.013121,-0.012114,-0.009983,-0.007534,-0.008408,-0.011158,-0.012987,5
4,0.000087,-0.000272,0.001022,0.003126,0.002284,0.000885,0.001933,0.002270,0.002247,0.002175,0.001750,0.001420,0.000297,0.000665,0.001911,0.001778,0.001087,-0.000716,-0.001347,-0.000384,-0.000135,0.000666,0.000656,0.000221,0.001470,0.001644,-0.000106,-0.001683,-0.001165,0.000241,0.000216,-0.